In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# base_folder = "/content/drive/MyDrive/Colab Notebooks/housing_fall2025"
# %cd "{base_folder}"

In [1]:
import sys
sys.executable

'c:\\Program Files\\Python310\\python.exe'

In [2]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from pathlib import Path
import os
import sqlite3
import pandas as pd
import tarfile
import urllib.request

BASE_DIR = Path("..")          # project root
DATA_DIR = BASE_DIR / "data"

def load_housing_data():
    print("[1] Checking for housing.tgz…")
    tarball_path = DATA_DIR / "housing.tgz"
    if not tarball_path.is_file():
        print("[1a] File not found. Creating datasets/ and downloading dataset…")
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        print("[1b] Download completed.")

    print("[2] Extracting housing.tgz…")
    with tarfile.open(tarball_path) as housing_tarball:
        housing_tarball.extractall(path=DATA_DIR)
    print("[3] Loading housing.csv into DataFrame…")
    return pd.read_csv(DATA_DIR / "housing" / "housing.csv")


def build_3nf_sqlite(db_path="housing.db"):
    print("=== BUILDING 3NF SQLITE DATA MODEL ===")

    print("\n[STEP 1] Loading CSV into DataFrame…")
    housing = load_housing_data()
    print(f"Loaded {len(housing)} rows.")

    print("\n[STEP 2] Creating surrogate key block_id…")
    housing = housing.reset_index().rename(columns={"index": "block_id"})
    print("block_id added.")

    print("\n[STEP 3] Building ocean_proximity dimension table…")
    ocean_dim = (
        housing[["ocean_proximity"]]
        .drop_duplicates()
        .reset_index(drop=True)
    )
    ocean_dim["ocean_proximity_id"] = ocean_dim.index + 1
    print(f"Found {len(ocean_dim)} unique ocean_proximity values.")

    print("\n[STEP 4] Merging ocean_proximity_id into main DataFrame…")
    housing = housing.merge(ocean_dim, on="ocean_proximity", how="left")

    print("\n[STEP 5] Creating 3NF DataFrames (ocean, block, stats)…")
    df_ocean = ocean_dim.rename(columns={"ocean_proximity": "name"})[
        ["ocean_proximity_id", "name"]
    ]
    df_block = housing[
        ["block_id", "longitude", "latitude", "ocean_proximity_id"]
    ].drop_duplicates(subset=["block_id"])
    df_stats = housing[
        [
            "block_id",
            "housing_median_age",
            "total_rooms",
            "total_bedrooms",
            "population",
            "households",
            "median_income",
            "median_house_value",
        ]
    ]
    print("3NF DataFrames created.")

    print("\n[STEP 6] Creating SQLite database and tables…")
    if os.path.exists(db_path):
        print("Existing DB found. Removing…")
        os.remove(db_path)

    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    print("Running SQL schema creation script…")
    cur.executescript(
        """
        DROP TABLE IF EXISTS block_housing_stats;
        DROP TABLE IF EXISTS block;
        DROP TABLE IF EXISTS ocean_proximity;

        CREATE TABLE ocean_proximity (
            ocean_proximity_id  INTEGER PRIMARY KEY,
            name                TEXT NOT NULL UNIQUE
        );

        CREATE TABLE block (
            block_id           INTEGER PRIMARY KEY,
            longitude          REAL NOT NULL,
            latitude           REAL NOT NULL,
            ocean_proximity_id INTEGER NOT NULL,
            FOREIGN KEY (ocean_proximity_id)
                REFERENCES ocean_proximity(ocean_proximity_id)
        );

        CREATE TABLE block_housing_stats (
            block_id            INTEGER PRIMARY KEY,
            housing_median_age  REAL NOT NULL,
            total_rooms         INTEGER NOT NULL,
            total_bedrooms      INTEGER,
            population          INTEGER NOT NULL,
            households          INTEGER NOT NULL,
            median_income       REAL NOT NULL,
            median_house_value  REAL NOT NULL,
            FOREIGN KEY (block_id)
                REFERENCES block(block_id)
        );
        """
    )
    print("Tables created.")

    print("\n[STEP 7] Inserting data into SQLite database…")
    print("Inserting ocean_proximity dimension…")
    cur.executemany(
        "INSERT INTO ocean_proximity (ocean_proximity_id, name) VALUES (?, ?)",
        list(df_ocean.itertuples(index=False, name=None)),
    )

    print("Inserting block table…")
    cur.executemany(
        """
        INSERT INTO block (block_id, longitude, latitude, ocean_proximity_id)
        VALUES (?, ?, ?, ?)
        """,
        list(df_block.itertuples(index=False, name=None)),
    )

    print("Inserting block_housing_stats…")
    cur.executemany(
        """
        INSERT INTO block_housing_stats (
            block_id,
            housing_median_age,
            total_rooms,
            total_bedrooms,
            population,
            households,
            median_income,
            median_house_value
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """,
        list(df_stats.itertuples(index=False, name=None)),
    )

    conn.commit()
    conn.close()

    print("\n=== DONE! SQLite DB created at:", db_path, "===\n")


build_3nf_sqlite("housing.db")


=== BUILDING 3NF SQLITE DATA MODEL ===

[STEP 1] Loading CSV into DataFrame…
[1] Checking for housing.tgz…
[2] Extracting housing.tgz…
[3] Loading housing.csv into DataFrame…
Loaded 20640 rows.

[STEP 2] Creating surrogate key block_id…
block_id added.

[STEP 3] Building ocean_proximity dimension table…
Found 5 unique ocean_proximity values.

[STEP 4] Merging ocean_proximity_id into main DataFrame…

[STEP 5] Creating 3NF DataFrames (ocean, block, stats)…
3NF DataFrames created.

[STEP 6] Creating SQLite database and tables…
Existing DB found. Removing…
Running SQL schema creation script…
Tables created.

[STEP 7] Inserting data into SQLite database…
Inserting ocean_proximity dimension…
Inserting block table…
Inserting block_housing_stats…

=== DONE! SQLite DB created at: housing.db ===



In [4]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("housing.db")

query = """
SELECT
    b.block_id,
    b.longitude,
    b.latitude,
    o.name AS ocean_proximity,
    s.housing_median_age,
    s.total_rooms,
    s.total_bedrooms,
    s.population,
    s.households,
    s.median_income,
    s.median_house_value
FROM block b
JOIN ocean_proximity o
    ON b.ocean_proximity_id = o.ocean_proximity_id
JOIN block_housing_stats s
    ON b.block_id = s.block_id
"""

df_joined = pd.read_sql_query(query, conn)
conn.close()

df_joined.head()


,block_id,longitude,latitude,ocean_proximity,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,0,-122.23,37.88,NEAR BAY,41.0,880,129.0,322,126,8.3252,452600.0
1,1,-122.22,37.86,NEAR BAY,21.0,7099,1106.0,2401,1138,8.3014,358500.0
2,2,-122.24,37.85,NEAR BAY,52.0,1467,190.0,496,177,7.2574,352100.0
3,3,-122.25,37.85,NEAR BAY,52.0,1274,235.0,558,219,5.6431,341300.0
4,4,-122.25,37.85,NEAR BAY,52.0,1627,280.0,565,259,3.8462,342200.0


In [5]:
df_joined.shape


(20640, 11)